In [1]:
from lightning import LitCNN
from utils.transforms import get_transform
from datasets import SmallDataset
from utils.all import collate_fn

from torch.utils import data as torch_data
from pytorch_lightning import Trainer
import math

/opt/conda/envs/eagles/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ssh -N -f -L localhost:16006:localhost:6006 kamranzolfonoon@35.203.109.50
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --port 6006

In [3]:
dataset = SmallDataset("/home/kamranzolfonoon/eagle-test-bucket", "/home/kamranzolfonoon/eagle-test-bucket/image_labels.json", get_transform(train=True))

# make training and test subsets
train_dataset = torch_data.Subset(dataset, range(0, math.floor(len(dataset) * 0.8)))
test_dataset = torch_data.Subset(dataset, range(math.floor(len(dataset) * 0.8), len(dataset)))

train_loader = torch_data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=1, collate_fn=collate_fn)
test_loader = torch_data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=1, collate_fn=collate_fn)

In [6]:
num_classes = 5

model_name = "fasterrcnn_resnet50_fpn"

model_params = {
    "num_classes": num_classes,
    "pretrained": True,
    "frozen": True
}

optimizer_name = "SGD"

optimizer_params = {
    "lr": 0.005,
    "momentum": 0.9,
    # "weight_decay": 0.0005
    "weight_decay": 0.0005
}

scheduler_name = "StepLR"

scheduler_params = {
    "step_size": 5,
    "gamma": 0.1
}

model = LitCNN(model_name, model_params, optimizer_name, optimizer_params, scheduler_name, scheduler_params)

In [9]:
for p in model.model.parameters():
    if p.requires_grad:
        print(p.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [5]:
trainer = Trainer(accelerator='gpu', devices=1, max_epochs=30)
trainer.fit(model, train_loader, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                 | Params
--------------------------------------------------------
0 | model          | FasterRCNN           | 41.3 M
1 | validation_map | MeanAveragePrecision | 0     
--------------------------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.279   Total estimated model params size (MB)


Epoch 0:   0%|          | 1/213 [00:00<01:35,  2.22it/s]                   

/opt/conda/envs/eagles/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 16:  20%|█▉        | 42/213 [00:26<01:49,  1.57it/s, loss=0.149, v_num=14] 

/opt/conda/envs/eagles/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
import torch
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.eval()
img, target = test_dataset[43]
img_cuda = img.to(device)
prediction = model([img_cuda])[0]
labels = []
boxes = []
print(prediction)
for i in range(len(prediction["labels"])):
    if prediction["scores"][i] > 0.5:
        labels.append(str(prediction["labels"][i].item()) + " " + str(round(prediction["scores"][i].item(), 3)))
        boxes.append(prediction["boxes"][i].cpu())
boxes = torch.stack(boxes)

box = draw_bounding_boxes(torch.tensor(img*255,dtype=torch.uint8), boxes=boxes,
                        labels=labels,
                        colors="red",
                        width=4,
                        font_size=100) 
im = to_pil_image(box.detach())
im.save("test.png")

{'boxes': tensor([[ 598.0652,  289.6666, 1055.9954,  598.5243],
        [ 471.4936,  426.0192,  610.3322,  526.6890],
        [ 908.2181,  535.2910, 1045.5083,  636.7701],
        [1037.4296,  258.6445, 1223.5970,  519.7628]], device='cuda:0',
       grad_fn=<StackBackward0>), 'labels': tensor([1, 2, 3, 1], device='cuda:0'), 'scores': tensor([0.9985, 0.9974, 0.9931, 0.9866], device='cuda:0',
       grad_fn=<IndexBackward0>)}


/tmp/ipykernel_922/634587109.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box = draw_bounding_boxes(torch.tensor(img*255,dtype=torch.uint8), boxes=boxes,
